In [ ]:
core_path=""
base_path=f'{core_path}/base'
credit_path=f'{core_path}/credit'

## 基本統計情報とデータ型

### credit_cards.csv - クレジットカード情報

- card_number: カード番号
- user_id: ユーザーID
- expiration_date: 有効期限
- card_type: カードの種類（Visa, MasterCardなど）

### card_transactions.csv - クレジットカード取引情報

- transaction_id: 取引ID
- card_number: カード番号
- transaction_amount: 取引額
- transaction_date: 取引日時
- store_id: 店舗ID


# 分析内容

次に、これらのデータを基に以下の分析を行います：  
1. クレジットカード情報の基本統計情報（カードタイプの分布、有効期限の分布など）
2. 取引データの基本統計情報（取引額の分布、取引の時間的傾向など）
3. ユーザーデータと取引データの結合による、ユーザー別の取引活動の分析

まずは、クレジットカード情報と取引データの基本統計情報を確認しましょう。

In [ ]:
import pandas as pd


# ユーザーデータのファイルを読み込む
user_data_path = f'{base_path}/users.csv'
user_data = pd.read_csv(user_data_path)

# データの最初の数行を表示して内容を確認
print(user_data.head())

# クレジットカードに関する情報のファイルを読み込む
credit_cards_path = f'{credit_path}/credit_cards.csv'
credit_cards_data = pd.read_csv(credit_cards_path)

# クレジットカード取引に関する情報のファイルを読み込む
card_transactions_path = f'{credit_path}/card_transactions.csv'
card_transactions_data = pd.read_csv(card_transactions_path)

# データの最初の数行を表示して内容を確認
credit_cards_data_head = credit_cards_data.head()
card_transactions_data_head = card_transactions_data.head()

print(credit_cards_data_head, card_transactions_data_head)


### クレジットカード情報 (credit_cards.csv)

- カードの総数は450枚です。
- カードタイプは5種類（Visa, MasterCard, JCB, Amex, Discover）で、Visaが111枚で最も多いです。
- 有効期限の分布は広がっていますが、特定の日付に集中する傾向は見られません。

### 取引データ (card_transactions.csv)

- 取引の総数は1000件です。
- 取引金額の平均は約24,828円、標準偏差は約14,394円です。
- 取引データは2022年12月12日から2023年12月12日までの1年間にわたっています。


### ユーザーデータと取引データの結合による、ユーザー別の取引活動の分析

次に、ユーザーデータと取引データを結合して、ユーザーごとの取引パターンや活動の分析を行います。これにより、どのようなユーザーセグメントがクレジットカードを頻繁に使用しているか、また取引額が大きいかなどの洞察が得られる可能性があります。  

結合にはユーザーID（user_id）を使用します。まずはカード情報と取引データをカード番号（card_number）で結合し、その後にユーザーデータを結合して分析を行います。 ​​  


In [ ]:
# クレジットカード情報の基本統計情報
credit_cards_info = credit_cards_data.describe(include='all')
credit_cards_types = credit_cards_data['card_type'].value_counts()

# 取引データの基本統計情報
card_transactions_info = card_transactions_data.describe()
card_transactions_data['transaction_date'] = pd.to_datetime(card_transactions_data['transaction_date'], errors='coerce')
transaction_date_distribution = card_transactions_data['transaction_date'].describe(datetime_is_numeric=True)

credit_cards_info, credit_cards_types, card_transactions_info, transaction_date_distribution



結合したデータから、クレジットカードの取引に関連するユーザー情報が見えてきました。このデータを利用して、以下の分析を行います：  
1. ユーザー別取引活動: 各ユーザーの取引件数、平均取引額、合計取引額など。
2. 年齢別取引傾向: 異なる年齢層のユーザーがどのような取引パターンを持っているか。
3. 時間的傾向: 取引が多い時間帯や曜日、月などの分析。

まずはユーザー別の取引活動に焦点を当てて分析を進めます。各ユーザーの取引件数、平均取引額、合計取引額を計算し、どのユーザーが最も活発にカードを使用しているかを把握しましょう。  

In [ ]:
# カード情報と取引データをカード番号で結合
combined_card_data = pd.merge(card_transactions_data, credit_cards_data, on="card_number")

# ユーザーデータと結合
combined_user_card_data = pd.merge(combined_card_data, user_data, on="user_id")

# 結合したデータの最初の数行を表示
combined_user_card_data.head()


ユーザー別のクレジットカード取引活動の分析結果は以下の通りです。  

最も活発なカード利用者の上位5名は以下のようになっています（ユーザーID順）：  

1. ユーザーID 448
   - 取引回数: 10回
   - 平均取引額: 約29,259円
   - 合計取引額: 292,585円
2. ユーザーID 379
   - 取引回数: 10回
   - 平均取引額: 約29,141円
   - 合計取引額: 291,414円
3. ユーザーID 44
   - 取引回数: 12回
   - 平均取引額: 約23,911円
   - 合計取引額: 286,934円
4. ユーザーID 394
   - 取引回数: 12回
   - 平均取引額: 約22,996円
   - 合計取引額: 275,947円
5. ユーザーID 193
   - 取引回数: 12回
   - 平均取引額: 約22,063円
   - 合計取引額: 264,751円

これらのデータから、TechCorpがどのような顧客層に対して特にサービスを強化するべきか、また特定の顧客層に向けたマーケティング戦略を策定する際の参考になるでしょう。

次に、年齢別取引傾向や時間的傾向についても分析していきます。これにより、年齢層や時間帯に応じた顧客行動の理解が深まります。どのような分析を希望されますか？

In [ ]:
# 各ユーザーの取引活動に関する統計情報を計算
user_transactions_stats = combined_user_card_data.groupby('user_id').agg(
    total_transactions=pd.NamedAgg(column='transaction_amount', aggfunc='count'),
    average_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='mean'),
    total_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='sum')
).reset_index()

# 統計情報を合計取引額で降順にソート
sorted_user_transactions_stats = user_transactions_stats.sort_values(by='total_transaction_amount', ascending=False)

sorted_user_transactions_stats.head()


次に、以下の2つの分析を進めます：  
1. **年齢別取引傾向**: 各年齢層のユーザーがどのような取引パターンを示しているかを分析します。これには取引件数、平均取引額、合計取引額を年齢層別に集計します。
2. **時間的傾向**: 取引データを時間別（時間帯、曜日、月）に分析して、特定の時間帯や期間における取引の傾向を把握します。

まずは年齢別取引傾向の分析から始めましょう。年齢層ごとの取引パターンを確認するために、年齢層別の統計情報を集計します。

年齢層別のクレジットカード取引活動の分析結果は以下の通りです。

| 年齢層 | 取引回数 | 平均取引額 (円) | 合計取引額 (円) |
| --- | --- | --- | --- |
| 10歳～20歳 | 61回 | 22,027 | 1,343,665 |
| 20歳～30歳 | 158回 | 24,076 | 3,804,030 |
| 30歳～40歳 | 203回 | 26,341 | 5,347,128 |
| 40歳～50歳 | 185回 | 25,738 | 4,761,492 |
| 50歳～60歳 | 211回 | 25,055 | 5,286,576 |
| 60歳～70歳 | 182回 | 23,546 | 4,285,370 |
| 70歳以上 | 0回 | - | 0 |

この分析から、30歳～60歳の年齢層がクレジットカード取引において最も活発であることがわかります。特に30歳～40歳と50歳～60歳の層は、取引回数と合計取引額が高いことが見て取れます。一方で、70歳以上の層では取引が確認されていません。  

次に、時間的傾向の分析に進みます。これには取引データを時間帯（朝/昼/夜）、曜日、月別に分析し、特定の時間帯や期間における取引の傾向を確認します。  

In [ ]:
# 年齢層別の取引活動に関する統計情報を計算
age_group_transactions_stats = combined_user_card_data.groupby(pd.cut(combined_user_card_data['age'], bins=range(10, 81, 10))).agg(
    total_transactions=pd.NamedAgg(column='transaction_amount', aggfunc='count'),
    average_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='mean'),
    total_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='sum')
).reset_index()

age_group_transactions_stats


時間的傾向に関するクレジットカード取引の分析結果は以下の通りです。  

### 時間帯別取引  

- 朝（Morning）: 261回の取引、平均取引額約25,048円
- 昼（Afternoon）: 244回の取引、平均取引額約25,285円
- 夕方（Evening）: 240回の取引、平均取引額約24,697円
- 深夜（Night）: 255回の取引、平均取引額約24,290円

### 曜日別取引
月曜（0）から日曜（6）までの取引は均等に分布しており、平均取引額は22,869円から25,967円の範囲です。    

### 月別取引
月別の取引件数は月によって若干の変動があり、平均取引額は21,925円から26,981円の範囲です。  

これらの分析から、時間帯、曜日、月別における取引の傾向が把握できます。時間帯別では、朝と昼にやや多くの取引があり、曜日別と月別の取引は比較的均等に分布しています。これらの情報は、特定の時間帯や曜日、月におけるプロモーションやマーケティング活動の計画に活用できます。

他に分析してほしい点や、次に進むデータがありましたらお知らせください。

In [ ]:
import numpy as np

# 時間帯、曜日、月のカラムを追加
combined_user_card_data['hour'] = combined_user_card_data['transaction_date'].dt.hour
combined_user_card_data['weekday'] = combined_user_card_data['transaction_date'].dt.weekday
combined_user_card_data['month'] = combined_user_card_data['transaction_date'].dt.month

# 時間帯を定義（朝: 6-12, 昼: 12-18, 夜: 18-24, 深夜: 0-6）
def assign_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour <= 23:
        return 'Evening'
    else:
        return 'Night'

combined_user_card_data['time_of_day'] = combined_user_card_data['hour'].apply(assign_time_of_day)

# 時間帯別、曜日別、月別の取引件数と平均取引額の計算
time_of_day_stats = combined_user_card_data.groupby('time_of_day').agg(
    total_transactions=pd.NamedAgg(column='transaction_amount', aggfunc='count'),
    average_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='mean')
)

weekday_stats = combined_user_card_data.groupby('weekday').agg(
    total_transactions=pd.NamedAgg(column='transaction_amount', aggfunc='count'),
    average_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='mean')
)

month_stats = combined_user_card_data.groupby('month').agg(
    total_transactions=pd.NamedAgg(column='transaction_amount', aggfunc='count'),
    average_transaction_amount=pd.NamedAgg(column='transaction_amount', aggfunc='mean')
)

time_of_day_stats, weekday_stats, month_stats
